In [1]:
import pymysql
from sqlalchemy import create_engine, text
from pandas import read_sql_query



from pydantic import (
    BaseSettings,
    Field
)

class Settings(BaseSettings):
    project_title: str = "Caris DATA API SERVICE"
    project_description: str = "REST API service for the datim reports and other relevant data analysis reports"
    project_version: str = "v0.0.1"
    project_docs_url: str = "/"
    
    DATIM_DESCRIPTION: str = "Get the datim data for dreams indicator - AGYW_PREV"
    DATIM_SUMMARY: str = "Get the datim data for dreams indicator - AGYW_PREV"

    DISTRICT_DESCRIPTION: str = "Get the district data for dreams indicator - AGYW_PREV"
    DISTRICT_SUMMARY: str = "Get the district data for dreams indicator - AGYW_PREV"
    
    
    mysql_username: str = Field(...,env='MYSQL_USERNAME')
    mysql_password: str = Field(...,env='MYSQL_PASSWORD')
    mysql_host: str = Field(...,env='MYSQL_HOST')
    mysql_port: int = Field(...,env='MYSQL_PORT')
    mysql_database: str = Field(...,env='MYSQL_DATABASE')
    
    commcare_username: str = Field(...,env='COMMCARE_USERNAME')
    commcare_password: str = Field(...,env='COMMCARE_PASSWORD')
    
    class Config:
        env_prefix = ""
        case_sensitive = False
        env_file = ".env"
        env_file_encoding = "utf-8"
        
        
        
settings = Settings()

from enum import Enum
from datetime import datetime


class Set_date(Enum):
    master_start = "2017-10-01"
    master_end = datetime.today().strftime('%Y-%m-%d')
    period_start = "2022-10-01"
    period_end = datetime.today().strftime('%Y-%m-%d')


QUERY_PERIOD = f"""
SELECT 
    a.id_patient,
    h.id_parenting_group,
    g.departement,
    g.commune,
    b.nbre_pres_for_inter,
    h.nbre_parenting_coupe_present,
    b.has_comdom_topic,
    b.has_preventive_vbg,
    d.number_of_condoms_sensibilize,
    d.number_condoms_sensibilization_date_in_the_interval,
    d.number_condoms_reception_in_the_interval,
    d.number_hiv_test_awareness_date_in_the_interval,
    d.type_of_test_vih,
    d.number_autotest_date_in_the_interval,
    d.number_test_date_in_the_interval,
    d.test_results,
    autotest_result,
    d.number_hiv_treatment_start_date_in_the_interval,
    d.number_vbg_treatment_date_in_the_interval,
    d.number_gynecological_care_date_in_the_interval,
    d.number_prep_awareness_date_in_the_interval,
    d.number_prep_reference_date_in_the_interval,
    d.number_prep_initiation_date_in_the_interval,
    d.number_contraceptive_sensibilization_date_in_the_interval,
    d.number_contraceptive_reception_in_the_interval,
    c.age_in_year,
    IF(c.age_in_year >= 10
            AND c.age_in_year <= 14,
        '10-14',
        IF(c.age_in_year >= 15
                AND c.age_in_year <= 19,
            '15-19',
            IF(c.age_in_year >= 20
                    AND c.age_in_year <= 24,
                '20-24',
                IF(c.age_in_year >= 25
                        AND c.age_in_year <= 29,
                    '25-29',
                    'not_valid_age')))) AS age_range,
    IF(c.age_in_year >= 10
            AND c.age_in_year <= 14,
        '10-14',
        IF(c.age_in_year >= 15
                AND c.age_in_year <= 17,
            '15-17',
            IF(c.age_in_year >= 18
                    AND c.age_in_year <= 24,
                '18-24',
                IF(c.age_in_year >= 25
                        AND c.age_in_year <= 29,
                    '25-29',
                    'not_valid_age')))) AS ovc_age,
    c.date_interview,
    IF(c.month_in_program >= 0
            AND c.month_in_program <= 6,
        '0-6 months',
        IF(c.month_in_program >= 7
                AND c.month_in_program <= 12,
            '07-12 months',
            IF(c.month_in_program >= 13
                    AND c.month_in_program <= 24,
                '13-24 months',
                '25+ months'))) AS month_in_program_range,
    IF(sc.id_patient IS NOT NULL,
        'yes',
        'no') AS has_schooling_payment_in_the_interval,
    IF(e.id_patient IS NOT NULL,
        'yes',
        'no') AS muso,
    IF(f.id_patient IS NOT NULL,
        'yes',
        'no') AS gardening,
    IF(past.id_patient IS NOT NULL,
        'yes',
        'no') AS has_a_service_with_date_in_the_past
FROM
    ((SELECT 
        dhi.id_patient
    FROM
        dream_hivinfos dhi
    WHERE
        (dhi.condom_sensibilization_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
            OR (dhi.contraceptive_reception_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
            OR (dhi.test_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
            OR (dhi.condoms_reception_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
            OR (dhi.vbg_treatment_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
            OR (dhi.gynecological_care_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
            OR (dhi.prep_initiation_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
            OR (dhi.has_been_sensibilize_for_condom = 1
            AND ((dhi.condom_sensibilization_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
            OR (dhi.condoms_reception_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')))) UNION (SELECT 
        dga.id_patient
    FROM
        dream_group_attendance dga
    LEFT JOIN dream_group_session dgs ON dgs.id = dga.id_group_session
    WHERE
        dga.value = 'P'
            AND dgs.date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}') UNION (SELECT 
        dpga.id_patient
    FROM
        dream_parenting_group_attendance dpga
    LEFT JOIN dream_parenting_group_session dpgs ON dpgs.id = dpga.id_parenting_group_session
    WHERE
        (dpga.parent_g = 'P'
            OR dpga.parent_vd = 'P'
            OR dpga.yg_g = 'P'
            OR dpga.yg_vd = 'P')
            AND dpgs.date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}') UNION (SELECT 
        dm.id_patient
    FROM
        dream_member dm
    INNER JOIN patient p ON p.id = dm.id_patient
    INNER JOIN muso_group_members mgm ON mgm.id_patient = dm.id_patient) UNION (SELECT 
        dmx.id_patient
    FROM
        dream_member dmx
    INNER JOIN patient px ON px.id = dmx.id_patient
    INNER JOIN gardening_beneficiary gbx ON gbx.code_dreams = px.patient_code
    GROUP BY dmx.id_patient) UNION (SELECT 
        ds.id_patient
    FROM
        caris_db.dreams_schooling ds
    WHERE
        ds.closed = FALSE AND ds.eskew_peye = 1
            AND (ds.dat_peyman_fet BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}'))) a
        LEFT JOIN
    (SELECT 
        xy.id_patient,
            COUNT(*) AS nbre_pres_for_inter,
            IF((SUM(number_of_session_s_08) > 0
                OR SUM(number_of_session_s_10) > 0
                OR SUM(number_of_session_s_11) > 0
                OR SUM(number_of_session_s_18) > 0), 'yes', 'no') AS has_comdom_topic,
            IF((SUM(number_of_session_s_14) > 0
                OR SUM(number_of_session_s_16) > 0), 'yes', 'no') AS has_preventive_vbg
    FROM
        (SELECT 
        id_patient,
            SUM(dgs.topic = 8) AS number_of_session_s_08,
            SUM(dgs.topic = 10) AS number_of_session_s_10,
            SUM(dgs.topic = 11) AS number_of_session_s_11,
            SUM(dgs.topic = 18) AS number_of_session_s_18,
            SUM(dgs.topic = 14) AS number_of_session_s_14,
            SUM(dgs.topic = 16) AS number_of_session_s_16
    FROM
        dream_group_attendance dga
    LEFT JOIN dream_group_session dgs ON dgs.id = dga.id_group_session
    WHERE
        dga.value = 'P'
            AND dgs.date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}'
    GROUP BY dga.id_patient , dgs.topic) xy
    GROUP BY xy.id_patient) b ON b.id_patient = a.id_patient
        LEFT JOIN
    (SELECT 
        dm2.id_patient,
            TIMESTAMPDIFF(MONTH, dsd.a1_dat_entvyou_a_ft_jjmmaa_egz_010817, NOW()) AS month_in_program,
            TIMESTAMPDIFF(YEAR, dsd.nan_ki_dat_ou_fet, NOW()) AS age_in_year,
            dsd.a1_dat_entvyou_a_ft_jjmmaa_egz_010817 AS date_interview
    FROM
        dream_member dm2
    LEFT JOIN dreams_surveys_data dsd ON dsd.case_id = dm2.case_id) c ON a.id_patient = c.id_patient
        LEFT JOIN
    (SELECT 
        dhi1.id_patient,
            SUM((dhi1.condom_sensibilization_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
                AND (dhi1.condom_sensibilization_date IS NOT NULL)) AS number_condoms_sensibilization_date_in_the_interval,
            SUM((dhi1.contraceptive_reception_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
                AND (dhi1.contraceptive_reception_date IS NOT NULL)) AS number_contraceptive_reception_in_the_interval,
            SUM(dhi1.has_been_sensibilize_for_condom = 1
                AND (dhi1.has_been_sensibilize_for_condom IS NOT NULL)) AS number_of_condoms_sensibilize,
            SUM((dhi1.condoms_reception_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
                AND (dhi1.condoms_reception_date IS NOT NULL)) AS number_condoms_reception_in_the_interval,
            SUM((dhi1.test_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
                AND (dhi1.test_date IS NOT NULL)) AS number_test_date_in_the_interval,
            SUM((dhi1.vbg_treatment_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
                AND (dhi1.vbg_treatment_date IS NOT NULL)) AS number_vbg_treatment_date_in_the_interval,
            SUM((dhi1.gynecological_care_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
                AND (dhi1.gynecological_care_date IS NOT NULL)) AS number_gynecological_care_date_in_the_interval,
            SUM((dhi1.prep_initiation_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
                AND (dhi1.prep_initiation_date IS NOT NULL)) AS number_prep_initiation_date_in_the_interval,
            GROUP_CONCAT(DISTINCT dhi1.test_result, ',') AS test_results,
            SUM((dhi1.prep_awareness_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
                AND (dhi1.prep_awareness_date IS NOT NULL)) AS number_prep_awareness_date_in_the_interval,
            SUM((dhi1.prep_reference_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
                AND (dhi1.prep_reference_date IS NOT NULL)) AS number_prep_reference_date_in_the_interval,
            SUM((dhi1.contraceptive_sensibilization_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
                AND (dhi1.contraceptive_sensibilization_date IS NOT NULL)) AS number_contraceptive_sensibilization_date_in_the_interval,
            SUM((dhi1.hiv_treatment_start_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
                AND (dhi1.hiv_treatment_start_date IS NOT NULL)) AS number_hiv_treatment_start_date_in_the_interval,
            SUM((dhi1.hiv_test_awareness_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
                AND (dhi1.hiv_test_awareness_date IS NOT NULL)) AS number_hiv_test_awareness_date_in_the_interval,
            GROUP_CONCAT(DISTINCT dhi1.type_of_test, ',') AS type_of_test_vih,
            SUM((dhi1.autotest_date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')
                AND (dhi1.autotest_date IS NOT NULL)) AS number_autotest_date_in_the_interval,
            GROUP_CONCAT(DISTINCT dhi1.autotest_result, ',') AS autotest_result
    FROM
        dream_hivinfos dhi1
    GROUP BY dhi1.id_patient) d ON a.id_patient = d.id_patient
        LEFT JOIN
    (SELECT 
        dm2.id_patient
    FROM
        dream_member dm2
    INNER JOIN muso_group_members mgm ON mgm.id_patient = dm2.id_patient
    GROUP BY dm2.id_patient) e ON a.id_patient = e.id_patient
        LEFT JOIN
    (SELECT 
        dm3.id_patient
    FROM
        dream_member dm3
    INNER JOIN patient p1 ON p1.id = dm3.id_patient
    INNER JOIN gardening_beneficiary gb ON gb.code_dreams = p1.patient_code
    GROUP BY dm3.id_patient) f ON a.id_patient = f.id_patient
        LEFT JOIN
    (SELECT 
        dmy.id_patient, lc.name AS commune, ld.name AS departement
    FROM
        dream_member dmy
    LEFT JOIN dream_group dg ON dg.id = dmy.id_group
    LEFT JOIN dream_hub dh ON dh.id = dg.id_dream_hub
    LEFT JOIN lookup_commune lc ON lc.id = dh.commune
    LEFT JOIN lookup_departement ld ON ld.id = lc.departement) g ON a.id_patient = g.id_patient
        LEFT JOIN
    (SELECT 
        dpga.id_patient,
            COUNT(*) AS nbre_parenting_coupe_present,
            dpgs.id_group AS id_parenting_group
    FROM
        dream_parenting_group_attendance dpga
    LEFT JOIN dream_parenting_group_session dpgs ON dpgs.id = dpga.id_parenting_group_session
    WHERE
        (dpga.parent_g = 'P'
            OR dpga.parent_vd = 'P'
            OR dpga.yg_g = 'P'
            OR dpga.yg_vd = 'P')
            AND dpgs.date BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}'
    GROUP BY id_patient) h ON h.id_patient = a.id_patient
    LEFT JOIN
    (SELECT 
        ds.id_patient
    FROM
        caris_db.dreams_schooling ds
    WHERE
        ds.closed = FALSE AND ds.eskew_peye = 1
        AND (ds.dat_peyman_fet BETWEEN '{Set_date.period_start.value}' AND '{Set_date.period_end.value}')) sc ON sc.id_patient = a.id_patient
        LEFT JOIN
    ((SELECT 
        dhi.id_patient
    FROM
        dream_hivinfos dhi
    WHERE
        (dhi.test_date < '{Set_date.period_start.value}')
            OR (dhi.condoms_reception_date < '{Set_date.period_start.value}')
            OR (dhi.vbg_treatment_date < '{Set_date.period_start.value}')
            OR (dhi.gynecological_care_date < '{Set_date.period_start.value}')
            OR (dhi.prep_initiation_date < '{Set_date.period_start.value}')
            OR (dhi.condom_sensibilization_date < '{Set_date.period_start.value}')
            OR (dhi.contraceptive_reception_date < '{Set_date.period_start.value}')) UNION (SELECT 
        dga.id_patient
    FROM
        dream_group_attendance dga
    LEFT JOIN dream_group_session dgs ON dgs.id = dga.id_group_session
    WHERE
        dga.value = 'P'
            AND dgs.date < '{Set_date.period_start.value}')) past ON past.id_patient = a.id_patient
"""


QUERY_MASTER = f"""
SELECT 
    a.id_patient,
    h.id_parenting_group,
    g.departement,
    g.commune,
    b.nbre_pres_for_inter,
    h.nbre_parenting_coupe_present,
    b.has_comdom_topic,
    b.has_preventive_vbg,
    d.number_of_condoms_sensibilize,
    d.number_condoms_sensibilization_date_in_the_interval,
    d.number_condoms_reception_in_the_interval,
    d.number_hiv_test_awareness_date_in_the_interval,
    d.type_of_test_vih,
    d.number_autotest_date_in_the_interval,
    d.number_test_date_in_the_interval,
    d.test_results,
    autotest_result,
    d.number_hiv_treatment_start_date_in_the_interval,
    d.number_vbg_treatment_date_in_the_interval,
    d.number_gynecological_care_date_in_the_interval,
    d.number_prep_awareness_date_in_the_interval,
    d.number_prep_reference_date_in_the_interval,
    d.number_prep_initiation_date_in_the_interval,
    d.number_contraceptive_sensibilization_date_in_the_interval,
    d.number_contraceptive_reception_in_the_interval,
    c.age_in_year,
    IF(c.age_in_year >= 10
            AND c.age_in_year <= 14,
        '10-14',
        IF(c.age_in_year >= 15
                AND c.age_in_year <= 19,
            '15-19',
            IF(c.age_in_year >= 20
                    AND c.age_in_year <= 24,
                '20-24',
                IF(c.age_in_year >= 25
                        AND c.age_in_year <= 29,
                    '25-29',
                    'not_valid_age')))) AS age_range,
    IF(c.age_in_year >= 10
            AND c.age_in_year <= 14,
        '10-14',
        IF(c.age_in_year >= 15
                AND c.age_in_year <= 17,
            '15-17',
            IF(c.age_in_year >= 18
                    AND c.age_in_year <= 24,
                '18-24',
                IF(c.age_in_year >= 25
                        AND c.age_in_year <= 29,
                    '25-29',
                    'not_valid_age')))) AS ovc_age,
    c.date_interview,
    IF(c.month_in_program >= 0
            AND c.month_in_program <= 6,
        '0-6 months',
        IF(c.month_in_program >= 7
                AND c.month_in_program <= 12,
            '07-12 months',
            IF(c.month_in_program >= 13
                    AND c.month_in_program <= 24,
                '13-24 months',
                '25+ months'))) AS month_in_program_range,
    IF(sc.id_patient IS NOT NULL,
        'yes',
        'no') AS has_schooling_payment_in_the_interval,
    IF(e.id_patient IS NOT NULL,
        'yes',
        'no') AS muso,
    IF(f.id_patient IS NOT NULL,
        'yes',
        'no') AS gardening,
    IF(past.id_patient IS NOT NULL,
        'yes',
        'no') AS has_a_service_with_date_in_the_past
FROM
    ((SELECT 
        dhi.id_patient
    FROM
        dream_hivinfos dhi
    WHERE
        (dhi.condom_sensibilization_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
            OR (dhi.contraceptive_reception_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
            OR (dhi.test_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
            OR (dhi.condoms_reception_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
            OR (dhi.vbg_treatment_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
            OR (dhi.gynecological_care_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
            OR (dhi.prep_initiation_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
            OR (dhi.has_been_sensibilize_for_condom = 1
            AND ((dhi.condom_sensibilization_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
            OR (dhi.condoms_reception_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')))) UNION (SELECT 
        dga.id_patient
    FROM
        dream_group_attendance dga
    LEFT JOIN dream_group_session dgs ON dgs.id = dga.id_group_session
    WHERE
        dga.value = 'P'
            AND dgs.date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}') UNION (SELECT 
        dpga.id_patient
    FROM
        dream_parenting_group_attendance dpga
    LEFT JOIN dream_parenting_group_session dpgs ON dpgs.id = dpga.id_parenting_group_session
    WHERE
        (dpga.parent_g = 'P'
            OR dpga.parent_vd = 'P'
            OR dpga.yg_g = 'P'
            OR dpga.yg_vd = 'P')
            AND dpgs.date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}') UNION (SELECT 
        dm.id_patient
    FROM
        dream_member dm
    INNER JOIN patient p ON p.id = dm.id_patient
    INNER JOIN muso_group_members mgm ON mgm.id_patient = dm.id_patient) UNION (SELECT 
        dmx.id_patient
    FROM
        dream_member dmx
    INNER JOIN patient px ON px.id = dmx.id_patient
    INNER JOIN gardening_beneficiary gbx ON gbx.code_dreams = px.patient_code
    GROUP BY dmx.id_patient) UNION (SELECT 
        ds.id_patient
    FROM
        caris_db.dreams_schooling ds
    WHERE
        ds.closed = FALSE AND ds.eskew_peye = 1
            AND (ds.dat_peyman_fet BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}'))) a
        LEFT JOIN
    (SELECT 
        xy.id_patient,
            COUNT(*) AS nbre_pres_for_inter,
            IF((SUM(number_of_session_s_08) > 0
                OR SUM(number_of_session_s_10) > 0
                OR SUM(number_of_session_s_11) > 0
                OR SUM(number_of_session_s_18) > 0), 'yes', 'no') AS has_comdom_topic,
            IF((SUM(number_of_session_s_14) > 0
                OR SUM(number_of_session_s_16) > 0), 'yes', 'no') AS has_preventive_vbg
    FROM
        (SELECT 
        id_patient,
            SUM(dgs.topic = 8) AS number_of_session_s_08,
            SUM(dgs.topic = 10) AS number_of_session_s_10,
            SUM(dgs.topic = 11) AS number_of_session_s_11,
            SUM(dgs.topic = 18) AS number_of_session_s_18,
            SUM(dgs.topic = 14) AS number_of_session_s_14,
            SUM(dgs.topic = 16) AS number_of_session_s_16
    FROM
        dream_group_attendance dga
    LEFT JOIN dream_group_session dgs ON dgs.id = dga.id_group_session
    WHERE
        dga.value = 'P'
            AND dgs.date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}'
    GROUP BY dga.id_patient , dgs.topic) xy
    GROUP BY xy.id_patient) b ON b.id_patient = a.id_patient
        LEFT JOIN
    (SELECT 
        dm2.id_patient,
            TIMESTAMPDIFF(MONTH, dsd.a1_dat_entvyou_a_ft_jjmmaa_egz_010817, NOW()) AS month_in_program,
            TIMESTAMPDIFF(YEAR, dsd.nan_ki_dat_ou_fet, NOW()) AS age_in_year,
            dsd.a1_dat_entvyou_a_ft_jjmmaa_egz_010817 AS date_interview
    FROM
        dream_member dm2
    LEFT JOIN dreams_surveys_data dsd ON dsd.case_id = dm2.case_id) c ON a.id_patient = c.id_patient
        LEFT JOIN
    (SELECT 
        dhi1.id_patient,
            SUM((dhi1.condom_sensibilization_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
                AND (dhi1.condom_sensibilization_date IS NOT NULL)) AS number_condoms_sensibilization_date_in_the_interval,
            SUM((dhi1.contraceptive_reception_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
                AND (dhi1.contraceptive_reception_date IS NOT NULL)) AS number_contraceptive_reception_in_the_interval,
            SUM(dhi1.has_been_sensibilize_for_condom = 1
                AND (dhi1.has_been_sensibilize_for_condom IS NOT NULL)) AS number_of_condoms_sensibilize,
            SUM((dhi1.condoms_reception_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
                AND (dhi1.condoms_reception_date IS NOT NULL)) AS number_condoms_reception_in_the_interval,
            SUM((dhi1.test_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
                AND (dhi1.test_date IS NOT NULL)) AS number_test_date_in_the_interval,
            SUM((dhi1.vbg_treatment_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
                AND (dhi1.vbg_treatment_date IS NOT NULL)) AS number_vbg_treatment_date_in_the_interval,
            SUM((dhi1.gynecological_care_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
                AND (dhi1.gynecological_care_date IS NOT NULL)) AS number_gynecological_care_date_in_the_interval,
            SUM((dhi1.prep_initiation_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
                AND (dhi1.prep_initiation_date IS NOT NULL)) AS number_prep_initiation_date_in_the_interval,
            GROUP_CONCAT(DISTINCT dhi1.test_result, ',') AS test_results,
            SUM((dhi1.prep_awareness_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
                AND (dhi1.prep_awareness_date IS NOT NULL)) AS number_prep_awareness_date_in_the_interval,
            SUM((dhi1.prep_reference_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
                AND (dhi1.prep_reference_date IS NOT NULL)) AS number_prep_reference_date_in_the_interval,
            SUM((dhi1.contraceptive_sensibilization_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
                AND (dhi1.contraceptive_sensibilization_date IS NOT NULL)) AS number_contraceptive_sensibilization_date_in_the_interval,
            SUM((dhi1.hiv_treatment_start_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
                AND (dhi1.hiv_treatment_start_date IS NOT NULL)) AS number_hiv_treatment_start_date_in_the_interval,
            SUM((dhi1.hiv_test_awareness_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
                AND (dhi1.hiv_test_awareness_date IS NOT NULL)) AS number_hiv_test_awareness_date_in_the_interval,
            GROUP_CONCAT(DISTINCT dhi1.type_of_test, ',') AS type_of_test_vih,
            SUM((dhi1.autotest_date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')
                AND (dhi1.autotest_date IS NOT NULL)) AS number_autotest_date_in_the_interval,
            GROUP_CONCAT(DISTINCT dhi1.autotest_result, ',') AS autotest_result
    FROM
        dream_hivinfos dhi1
    GROUP BY dhi1.id_patient) d ON a.id_patient = d.id_patient
        LEFT JOIN
    (SELECT 
        dm2.id_patient
    FROM
        dream_member dm2
    INNER JOIN muso_group_members mgm ON mgm.id_patient = dm2.id_patient
    GROUP BY dm2.id_patient) e ON a.id_patient = e.id_patient
        LEFT JOIN
    (SELECT 
        dm3.id_patient
    FROM
        dream_member dm3
    INNER JOIN patient p1 ON p1.id = dm3.id_patient
    INNER JOIN gardening_beneficiary gb ON gb.code_dreams = p1.patient_code
    GROUP BY dm3.id_patient) f ON a.id_patient = f.id_patient
        LEFT JOIN
    (SELECT 
        dmy.id_patient, lc.name AS commune, ld.name AS departement
    FROM
        dream_member dmy
    LEFT JOIN dream_group dg ON dg.id = dmy.id_group
    LEFT JOIN dream_hub dh ON dh.id = dg.id_dream_hub
    LEFT JOIN lookup_commune lc ON lc.id = dh.commune
    LEFT JOIN lookup_departement ld ON ld.id = lc.departement) g ON a.id_patient = g.id_patient
        LEFT JOIN
    (SELECT 
        dpga.id_patient,
            COUNT(*) AS nbre_parenting_coupe_present,
            dpgs.id_group AS id_parenting_group
    FROM
        dream_parenting_group_attendance dpga
    LEFT JOIN dream_parenting_group_session dpgs ON dpgs.id = dpga.id_parenting_group_session
    WHERE
        (dpga.parent_g = 'P'
            OR dpga.parent_vd = 'P'
            OR dpga.yg_g = 'P'
            OR dpga.yg_vd = 'P')
            AND dpgs.date BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}'
    GROUP BY id_patient) h ON h.id_patient = a.id_patient
    LEFT JOIN
    (SELECT 
        ds.id_patient
    FROM
        caris_db.dreams_schooling ds
    WHERE
        ds.closed = FALSE AND ds.eskew_peye = 1
        AND (ds.dat_peyman_fet BETWEEN '{Set_date.master_start.value}' AND '{Set_date.master_end.value}')) sc ON sc.id_patient = a.id_patient
        LEFT JOIN
    ((SELECT 
        dhi.id_patient
    FROM
        dream_hivinfos dhi
    WHERE
        (dhi.test_date < '{Set_date.master_start.value}')
            OR (dhi.condoms_reception_date < '{Set_date.master_start.value}')
            OR (dhi.vbg_treatment_date < '{Set_date.master_start.value}')
            OR (dhi.gynecological_care_date < '{Set_date.master_start.value}')
            OR (dhi.prep_initiation_date < '{Set_date.master_start.value}')
            OR (dhi.condom_sensibilization_date < '{Set_date.master_start.value}')
            OR (dhi.contraceptive_reception_date < '{Set_date.master_start.value}')) UNION (SELECT 
        dga.id_patient
    FROM
        dream_group_attendance dga
    LEFT JOIN dream_group_session dgs ON dgs.id = dga.id_group_session
    WHERE
        dga.value = 'P'
            AND dgs.date < '{Set_date.master_start.value}')) past ON past.id_patient = a.id_patient
"""













engine = create_engine(
    f"mysql+pymysql://{settings.mysql_username}:{settings.mysql_password}@{settings.mysql_host}/{settings.mysql_database}")


agyw_served_period = read_sql_query(text(QUERY_PERIOD), engine.connect(), parse_dates=True)
agyw_served = read_sql_query(text(QUERY_MASTER), engine.connect(), parse_dates=True)

# close the pool of connection
engine.dispose()


In [2]:
from numpy import int16


def type_test(tt):
    if tt == '0,':
        return "sanguin"
    elif ((tt == '1,') | (tt == '0,,1,')):
        return 'autotest'
    elif(tt == 'no'):
        return 'no_info'
    else:
        return 'verify_me'


def vih_autotest_result(var):
    if var == 'no':
        return 'no_info'
    elif(
        var == '0,'
    ):
        return 'indeterminee'
    elif(
        (var == '0,,1,') |
        (var == '1,')
    ):
        return 'non_reactif'
    elif(
        (var == '2,') |
        (var == '0,,2,')
    ):
        return 'reactif'
    else:
        'verify_me'


def vih_test_result(vtr):
    if vtr == '0,':
        return 'no_info'
    elif(
        (vtr == '0,,3,') |
        (vtr == '0,,2,,3,') |
        (vtr == '2,,3,') |
        (vtr == '3,')
    ):
        return 'indetermine'
    elif(
        (vtr == '0,,2,') |
        (vtr == '2,')
    ):
        return 'negatif'
    elif(
        (vtr == '0,,1,') |
        (vtr == '1,')
    ):
        return 'positif'
    else:
        return 'verify_me'


def curriculum_detailed(pres):
    if pres >= 17:
        return "yes"
    elif 1 <= pres <= 16:
        return "has_started"
    else:
        return "no"


def curriculum(pres):
    return "yes" if pres >= 17 else "no"


def parenting_detailed(pres):
    if pres >= 12:
        return "yes"
    elif 1 <= pres <= 11:
        return "has_started"
    else:
        return "no"


def parenting(pres):
    return "yes" if pres >= 12 else "no"


def condom(df):
    return "yes" if (df.has_comdom_topic == "yes" or df.number_of_condoms_sensibilize > 0 or df.number_condoms_reception_in_the_interval > 0 or df.number_condoms_sensibilization_date_in_the_interval > 0) else "no"


def hts_awareness(ha):
    return 'yes' if ha > 0 else 'no'


def treatment_debut(tdebut):
    return 'yes' if tdebut > 0 else 'no'


def hts(hd):
    return "yes" if hd > 0 else "no"


def vbg(vbg):
    return "yes" if vbg > 0 else "no"


def gyneco(gyneco):
    return "yes" if gyneco > 0 else "no"


def prep_awareness(pa):
    return "yes" if pa > 0 else "no"


def prep_reference(pr):
    return "yes" if pr > 0 else "no"


def prep(pd):
    return "yes" if pd > 0 else "no"


def contraceptive_awareness(caw):
    return 'yes' if caw > 0 else 'no'


def contraceptive(cd):
    return "yes" if cd > 0 else "no"


def postcare(df):
    return "yes" if (df.number_vbg_treatment_date_in_the_interval > 0 or df.number_gynecological_care_date_in_the_interval > 0) else "no"


def socioeco(df):
    return "yes" if ((df.muso == "yes") or (df.gardening == "yes")) else "no"


def prim_1014(df):
    return "primary" if (df.age_range == "10-14" and df.curriculum == "yes") else "no"


def prim_1519(df):
    return "primary" if (df.age_range == "15-19" and df.curriculum == "yes" and df.condom == "yes") else "no"


def prim_2024(df):
    return "primary" if (df.age_range == "20-24" and df.curriculum == "yes" and df.condom == "yes") else "no"


"""
def sec_1014(df):
    return "secondary" if (df.age_range=="10-14" and ((df.condom=="yes")|(df.hts=="yes")|(df.post_violence_care=="yes")|(df.socioeco_app=="yes")|(df.prep=="yes"))) else "no"

def sec_1519(df):
    return "secondary" if (df.age_range=="15-19" and ((df.hts=="yes")|(df.post_violence_care=="yes")|(df.socioeco_app=="yes")|(df.prep=="yes"))) else "no"

def sec_2024(df):
    return "secondary" if (df.age_range=="20-24" and ((df.hts=="yes")|(df.post_violence_care=="yes")|(df.socioeco_app=="yes")|(df.prep=="yes"))) else "no"



def comp_1014(df):
    return "complete" if (df.age_range=="10-14" and  df.curriculum=="no" and ((df.condom=="yes")|(df.hts=="yes")|(df.post_violence_care=="yes")|(df.socioeco_app=="yes")|(df.prep=="yes"))) else "no"

def comp_1519(df):
    return "complete" if (df.age_range=="15-19" and  (((df.curriculum=="yes")&(df.condom=="no"))|((df.curriculum=="no")&(df.condom=="yes"))) and ((df.hts=="yes")|(df.post_violence_care=="yes")|(df.socioeco_app=="yes")|(df.prep=="yes"))) else "no"

def comp_2024(df):
    return "complete" if (df.age_range=="20-24" and  (((df.curriculum=="yes")&(df.condom=="no"))|((df.curriculum=="no")&(df.condom=="yes"))) and ((df.hts=="yes")|(df.post_violence_care=="yes")|(df.socioeco_app=="yes")|(df.prep=="yes"))) else "no"

"""

'\ndef sec_1014(df):\n    return "secondary" if (df.age_range=="10-14" and ((df.condom=="yes")|(df.hts=="yes")|(df.post_violence_care=="yes")|(df.socioeco_app=="yes")|(df.prep=="yes"))) else "no"\n\ndef sec_1519(df):\n    return "secondary" if (df.age_range=="15-19" and ((df.hts=="yes")|(df.post_violence_care=="yes")|(df.socioeco_app=="yes")|(df.prep=="yes"))) else "no"\n\ndef sec_2024(df):\n    return "secondary" if (df.age_range=="20-24" and ((df.hts=="yes")|(df.post_violence_care=="yes")|(df.socioeco_app=="yes")|(df.prep=="yes"))) else "no"\n\n\n\ndef comp_1014(df):\n    return "complete" if (df.age_range=="10-14" and  df.curriculum=="no" and ((df.condom=="yes")|(df.hts=="yes")|(df.post_violence_care=="yes")|(df.socioeco_app=="yes")|(df.prep=="yes"))) else "no"\n\ndef comp_1519(df):\n    return "complete" if (df.age_range=="15-19" and  (((df.curriculum=="yes")&(df.condom=="no"))|((df.curriculum=="no")&(df.condom=="yes"))) and ((df.hts=="yes")|(df.post_violence_care=="yes")|(df.socio

In [3]:
from dataclasses import dataclass, InitVar

@dataclass
class AGYW_Analysis:
    def __init__(self, agyw_served, agyw_served_period):
        self.agyw_served:InitVar = agyw_served
        self.agyw_served_period: InitVar = agyw_served_period
        self.__actif_served = agyw_served[agyw_served.id_patient.isin(
    agyw_served_period.id_patient)]
         
    def data_actif_served(self):
        __actif_served = self.__actif_served
        __actif_served.type_of_test_vih.fillna('no', inplace=True)
        __actif_served.test_results.fillna('0,', inplace=True)
        __actif_served.autotest_result.fillna('no', inplace=True)

        __actif_served.nbre_pres_for_inter.fillna(0, inplace=True)
        __actif_served.has_comdom_topic.fillna('no', inplace=True)
        __actif_served.number_of_condoms_sensibilize.fillna(0, inplace=True)
        __actif_served.number_condoms_reception_in_the_interval.fillna(0, inplace=True)
        __actif_served.number_test_date_in_the_interval.fillna(0, inplace=True)
        __actif_served.number_gynecological_care_date_in_the_interval.fillna(
            0, inplace=True)
        __actif_served.number_vbg_treatment_date_in_the_interval.fillna(0, inplace=True)
        __actif_served.number_prep_initiation_date_in_the_interval.fillna(
            0, inplace=True)
        __actif_served.nbre_parenting_coupe_present.fillna(0, inplace=True)
        __actif_served.number_contraceptive_reception_in_the_interval.fillna(
            0, inplace=True)
        __actif_served.number_condoms_sensibilization_date_in_the_interval.fillna(
            0, inplace=True)
        __actif_served.number_prep_awareness_date_in_the_interval.fillna(0, inplace=True)
        __actif_served.number_prep_reference_date_in_the_interval.fillna(0, inplace=True)
        __actif_served.number_hiv_test_awareness_date_in_the_interval.fillna(
            0, inplace=True)
        __actif_served.number_hiv_treatment_start_date_in_the_interval.fillna(
            0, inplace=True)
        __actif_served.number_contraceptive_sensibilization_date_in_the_interval.fillna(
            0, inplace=True)

        __actif_served.nbre_pres_for_inter = __actif_served.nbre_pres_for_inter.astype(
            int16)
        __actif_served.number_of_condoms_sensibilize = __actif_served.number_of_condoms_sensibilize.astype(
            int16)
        __actif_served.number_condoms_reception_in_the_interval = __actif_served.number_condoms_reception_in_the_interval.astype(
            int16)
        __actif_served.number_test_date_in_the_interval = __actif_served.number_test_date_in_the_interval.astype(
            int16)
        __actif_served.number_gynecological_care_date_in_the_interval = __actif_served.number_gynecological_care_date_in_the_interval.astype(
            int16)
        __actif_served.number_vbg_treatment_date_in_the_interval = __actif_served.number_vbg_treatment_date_in_the_interval.astype(
            int16)
        __actif_served.number_prep_initiation_date_in_the_interval = __actif_served.number_prep_initiation_date_in_the_interval.astype(
            int16)
        __actif_served.nbre_parenting_coupe_present = __actif_served.nbre_parenting_coupe_present.astype(
            int16)
        __actif_served.number_contraceptive_reception_in_the_interval = __actif_served.number_contraceptive_reception_in_the_interval.astype(
            int16)
        __actif_served.number_condoms_sensibilization_date_in_the_interval = __actif_served.number_condoms_sensibilization_date_in_the_interval.astype(
            int16)
        __actif_served.number_prep_awareness_date_in_the_interval = __actif_served.number_prep_awareness_date_in_the_interval.astype(
            int16)
        __actif_served.number_prep_reference_date_in_the_interval = __actif_served.number_prep_reference_date_in_the_interval.astype(
            int16)
        __actif_served.number_hiv_test_awareness_date_in_the_interval = __actif_served.number_hiv_test_awareness_date_in_the_interval.astype(
            int16)
        __actif_served.number_hiv_treatment_start_date_in_the_interval = __actif_served.number_hiv_treatment_start_date_in_the_interval.astype(
            int16)
        __actif_served.number_contraceptive_sensibilization_date_in_the_interval = __actif_served.number_contraceptive_sensibilization_date_in_the_interval.astype(
            int16)

        # services
        __actif_served['education'] = __actif_served.has_schooling_payment_in_the_interval
        __actif_served['parenting_detailed'] = __actif_served.nbre_parenting_coupe_present.map(
            parenting_detailed)
        __actif_served['parenting'] = __actif_served.nbre_parenting_coupe_present.map(
            parenting)

        __actif_served['curriculum_detailed'] = __actif_served.nbre_pres_for_inter.map(
            curriculum_detailed)
        __actif_served['curriculum'] = __actif_served.nbre_pres_for_inter.map(curriculum)

        __actif_served['condom'] = __actif_served.apply(lambda df: condom(df), axis=1)

        __actif_served['hts_awareness'] = __actif_served.number_hiv_test_awareness_date_in_the_interval.map(
            hts_awareness)
        __actif_served['hts_type_test'] = __actif_served.type_of_test_vih.map(type_test)
        __actif_served['hts'] = __actif_served.number_test_date_in_the_interval.map(hts)
        __actif_served['hts_autotest_result'] = __actif_served.autotest_result.map(
            vih_autotest_result)
        __actif_served['hts_test_result'] = __actif_served.test_results.map(
            vih_test_result)
        __actif_served['hts_treatment_debut'] = __actif_served.number_hiv_treatment_start_date_in_the_interval.map(
            treatment_debut)


        __actif_served['vbg'] = __actif_served.number_vbg_treatment_date_in_the_interval.map(
            vbg)
        __actif_served['gyneco'] = __actif_served.number_gynecological_care_date_in_the_interval.map(
            gyneco)

        __actif_served['post_violence_care'] = __actif_served.apply(
            lambda df: postcare(df), axis=1)

        __actif_served['socioeco_app'] = __actif_served.apply(
            lambda df: socioeco(df), axis=1)

        __actif_served['prep_awareness'] = __actif_served.number_prep_awareness_date_in_the_interval.map(
            prep_awareness)
        __actif_served['prep_reference'] = __actif_served.number_prep_reference_date_in_the_interval.map(
            prep_reference)
        __actif_served['prep'] = __actif_served.number_prep_initiation_date_in_the_interval.map(
            prep)

        __actif_served['contraceptive_awareness'] = __actif_served.number_contraceptive_sensibilization_date_in_the_interval.map(
            contraceptive_awareness)
        __actif_served['contraceptive'] = __actif_served.number_contraceptive_reception_in_the_interval.map(
            contraceptive)

        __actif_served['ps_1014'] = __actif_served.apply(lambda df: prim_1014(df), axis=1)
        __actif_served['ps_1519'] = __actif_served.apply(lambda df: prim_1519(df), axis=1)
        __actif_served['ps_2024'] = __actif_served.apply(lambda df: prim_2024(df), axis=1)
        return self.__actif_served

In [4]:
AGYW_ACTIF = AGYW_Analysis(agyw_served,agyw_served_period).data_actif_served()
AGYW_ACTIF

/var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/ipykernel_54471/37542156.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  __actif_served.type_of_test_vih.fillna('no', inplace=True)
/var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/ipykernel_54471/37542156.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  __actif_served.test_results.fillna('0,', inplace=True)
/var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/ipykernel_54471/37542156.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/us

,id_patient,id_parenting_group,departement,commune,nbre_pres_for_inter,nbre_parenting_coupe_present,has_comdom_topic,has_preventive_vbg,number_of_condoms_sensibilize,number_condoms_sensibilization_date_in_the_interval,...,post_violence_care,socioeco_app,prep_awareness,prep_reference,prep,contraceptive_awareness,contraceptive,ps_1014,ps_1519,ps_2024
226,75394,3363.0,Nord,Milot,21,10,yes,yes,0,0,...,no,yes,no,no,no,no,no,no,primary,no
227,75395,NaN,Nord,Milot,21,0,yes,yes,0,0,...,no,yes,no,no,no,no,no,no,primary,no
244,75528,NaN,Nord,Cap-Haïtien,17,0,yes,yes,1,0,...,no,yes,no,no,no,no,no,no,no,primary
275,75987,NaN,Artibonite,Dessalines,17,0,yes,yes,1,0,...,no,yes,yes,no,no,no,no,no,no,primary
287,75964,NaN,Artibonite,Liancourt,17,0,yes,yes,0,0,...,no,yes,no,no,no,no,no,no,no,primary
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49664,101428,NaN,Nord,Milot,0,0,no,None,0,0,...,no,yes,no,no,no,no,no,no,no,no
49665,104847,NaN,Artibonite,Desdunes,0,0,no,None,0,0,...,no,yes,no,no,no,no,no,no,no,no
49666,105142,NaN,Nord,Cap-Haïtien,0,0,no,None,0,0,...,no,yes,no,no,no,no,no,no,no,no
49667,105158,NaN,Nord,Cap-Haïtien,0,0,no,None,0,0,...,no,yes,no,no,no,no,no,no,no,no


In [5]:
from pandas import DataFrame

class AgywPrev:
    """A class with properties and methods given the results of the indicator AGYW_PREV DATIM"""
    __who_am_I = "DATIM"
    __datim1_title = "Number of individual AGYW that have fully completed the entire DREAMS primary package of services/interventions but no additional services/interventions."
    __datim2_title = "Number of individual AGYW that have fully completed the entire DREAMS primary package of services/interventions AND at least one secondary service/intervention."
    __datim3_title = "Number of individual AGYW that have completed at least one DREAMS service/intervention but not the full primary package."
    __datim4_title = "Number of AGYW that have started a DREAMS service/intervention but have not yet completed it."

    def __repr__(self):
        return f"<AgywPrev {self.__i_am}>"

    def __str__(self):
        return f"<AgywPrev {self.__i_am}>"

    @classmethod
    def datim_titleI(cls):
        return cls.__datim1_title

    @classmethod
    def datim_titleII(cls):
        return cls.__datim2_title

    @classmethod
    def datim_titleIII(cls):
        return cls.__datim3_title

    @classmethod
    def datim_titleIV(cls):
        return cls.__datim4_title

    def __init__(self, commune=None,data=None):
        self.__commune = commune
        self.__i_am = f"{AgywPrev.__who_am_I}"
        self.__data = data
        self. __total_mastersheet = self.__data.id_patient.count()
        if self.__commune == None:
            self.__dreams_valid = self.__data[(self.__data.age_range != "not_valid_age") & (
                self.__data.age_range != "25-29")]
        else:
            self.__dreams_valid = self.__data[(self.__data.age_range != "not_valid_age") & (
                self.__data.age_range != "25-29") & (self.__data.commune == f"{self.__commune}")]
        self.__total_dreams_valid = self.__dreams_valid.id_patient.count()
        self.__dreams_valid["primary_only"] = self.__dreams_valid.apply(
            lambda df: self.__primFunc(df), axis=1)
        self.__dreams_valid["primary_and_OneSecondary_services"] = self.__dreams_valid.apply(
            lambda df: self.__primLeastOneSecFunc(df), axis=1)
        self.__dreams_valid["completed_one_service"] = self.__dreams_valid.apply(
            lambda df: self.__primPartFunc(df), axis=1)
        self.__dreams_valid["has_started_one_service"] = self.__dreams_valid.apply(
            lambda df: self.__hasStartedFunc(df), axis=1)
        self.__agyw_prevI = self.__dreams_valid[self.__dreams_valid.primary_only ==
                                                "full_primary_only"]
        self.__agyw_prevII = self.__dreams_valid[self.__dreams_valid.primary_and_OneSecondary_services ==
                                                 "full_primary_leastOneSecondary"]
        self.__agyw_prevIII = self.__dreams_valid[self.__dreams_valid.completed_one_service ==
                                                  "primary_part_services"]
        self.__agyw_prevIV = self.__dreams_valid[self.__dreams_valid.has_started_one_service == "yes"]
        self.__agyw_prevI_total = self.__agyw_prevI.id_patient.count()
        self.__agyw_prevII_total = self.__agyw_prevII.id_patient.count()
        self.__agyw_prevIII_total = self.__agyw_prevIII.id_patient.count()
        self.__agyw_prevIV_total = self.__agyw_prevIV.id_patient.count()
        self.__total_datim = self.__agyw_prevI_total + self.__agyw_prevII_total + \
            self.__agyw_prevIII_total + self.__agyw_prevIV_total

    @property
    def who_am_i(self):
        return self.__i_am

    @property
    def data_mastersheet(self):
        return self.__data

    @property
    def data_dreams_valid(self):
        return self.__dreams_valid

    @property
    def total_mastersheet(self):
        return self.__total_mastersheet

    @property
    def total_dreams_valid(self):
        return self.__total_dreams_valid

    def __primFunc(self, df):
        if (df.ps_1014 == "primary" and df.hts == "no" and df.prep == "no" and df.condom == "no" and df.post_violence_care == "no" and df.socioeco_app == "no" and df.parenting == "no" and df.contraceptive == "no"):
            return "full_primary_only"
        elif (df.ps_1519 == "primary" and df.hts == "no" and df.prep == "no" and df.post_violence_care == "no" and df.socioeco_app == "no" and df.parenting == "no" and df.contraceptive == "no"):
            return "full_primary_only"
        elif (df.ps_2024 == "primary" and df.hts == "no" and df.prep == "no" and df.post_violence_care == "no" and df.socioeco_app == "no" and df.parenting == "no" and df.contraceptive == "no"):
            return "full_primary_only"
        else:
            return "invalid"

    def __primLeastOneSecFunc(self, df):
        if (df.ps_1014 == "primary") and (df.hts == "yes" or df.prep == "yes" or df.condom == "yes" or df.post_violence_care == "yes" or df.socioeco_app == "yes" or df.parenting == "yes" or df.contraceptive == "yes"):
            return "full_primary_leastOneSecondary"
        elif (df.ps_1519 == "primary") and (df.hts == "yes" or df.prep == "yes" or df.post_violence_care == "yes" or df.socioeco_app == "yes" or df.parenting == "yes" or df.contraceptive == "yes"):
            return "full_primary_leastOneSecondary"
        elif (df.ps_2024 == "primary") and (df.hts == "yes" or df.prep == "yes" or df.post_violence_care == "yes" or df.socioeco_app == "yes" or df.parenting == "yes" or df.contraceptive == "yes"):
            return "full_primary_leastOneSecondary"
        else:
            return "invalid"

    def __primPartFunc(self, df):
        if (df.age_range == "10-14") and (df.primary_only == "invalid") and (df.primary_and_OneSecondary_services == 'invalid') and ((df.hts == "yes" or df.prep == "yes" or df.condom == "yes" or df.post_violence_care == "yes" or df.socioeco_app == "yes" or df.parenting == "yes" or df.contraceptive == "yes")):
            return "primary_part_services"
        elif (df.age_range == "15-19") and (df.primary_only == "invalid") and (df.primary_and_OneSecondary_services == 'invalid') and (df.curriculum == "yes" or df.condom == "yes" or df.hts == "yes" or df.prep == "yes" or df.post_violence_care == "yes" or df.socioeco_app == "yes" or df.parenting == "yes" or df.contraceptive == "yes"):
            return "primary_part_services"
        elif (df.age_range == "20-24") and (df.primary_only == "invalid") and (df.primary_and_OneSecondary_services == 'invalid') and ((df.curriculum == "yes" or df.condom == "yes" or df.hts == "yes" or df.prep == "yes" or df.post_violence_care == "yes" or df.socioeco_app == "yes" or df.parenting == "yes" or df.contraceptive == "yes")):
            return "primary_part_services"
        else:
            return "invalid"

    def __hasStartedFunc(self, df):
        if (df.age_range == "10-14") and (df.primary_only == "invalid") and (df.primary_and_OneSecondary_services == 'invalid') and (df.completed_one_service == "invalid"):
            return "yes"
        elif (df.age_range == "15-19") and (df.primary_only == "invalid") and (df.primary_and_OneSecondary_services == 'invalid') and (df.completed_one_service == "invalid"):
            return "yes"
        elif (df.age_range == "20-24") and (df.primary_only == "invalid") and (df.primary_and_OneSecondary_services == 'invalid') and (df.completed_one_service == "invalid"):
            return "yes"
        else:
            return "no"

    @property
    def total_datimI(self):
        return self.__agyw_prevI_total

    @property
    def total_datimII(self):
        return self.__agyw_prevII_total

    @property
    def total_datimIII(self):
        return self.__agyw_prevIII_total

    @property
    def total_datimIV(self):
        return self.__agyw_prevIV_total

    @property
    def total_datim_general(self):
        return self.__total_datim

    @property
    def data_agyw_prevI(self):
        return self.__agyw_prevI

    @property
    def data_agyw_prevII(self):
        return self.__agyw_prevII

    @property
    def data_agyw_prevIII(self):
        return self.__agyw_prevIII

    @property
    def data_agyw_prevIV(self):
        return self.__agyw_prevIV


    #__PERIOD_DATIM = sorted(list(a.month_in_program_range.unique()))
    #__PERIOD_DATIM.append("Total")
    #__AGE_DATIM = sorted(list(agyw_actif().age_range.unique()))[0:3]

    def datim_agyw_prevI(self):
        __PERIOD_DATIM = sorted(list(self.__data.month_in_program_range.unique()))
        __PERIOD_DATIM.append("Total")
        __AGE_DATIM = sorted(list(self.__data.age_range.unique()))[0:3]

        try:
            pivotableI = self.__agyw_prevI.rename(
                columns={"age_range": "Age", "month_in_program_range": "Time"})
            agyw_prevI_pivot = pivotableI.pivot_table(index="Age", columns="Time", values="id_patient",
                                                      aggfunc="count", fill_value=0, margins=True, margins_name="Total", dropna=False)[:-1]
            columns_pivotI = list(agyw_prevI_pivot.columns)
            indexes_pivotI = list(agyw_prevI_pivot.index)
            for period in __PERIOD_DATIM:
                if period not in columns_pivotI:
                    agyw_prevI_pivot[period] = 0
            for age in __AGE_DATIM:
                if age not in indexes_pivotI:
                    agyw_prevI_pivot.loc[age] = 0
            agyw_prevI_pivot = agyw_prevI_pivot.reindex(
                index=__AGE_DATIM, columns=__PERIOD_DATIM)
            agyw_prevI_pivot_final = agyw_prevI_pivot.reset_index().rename_axis(None, axis=1)
            agyw_prevI_results_final = DataFrame(
                agyw_prevI_pivot_final.to_records(index=False))
        except ValueError:
            agyw_prevI_results_final = DataFrame({"Age": ["10-14", "15-19",
                                                          "20-24"],
                                                  "0-6 months": [0, 0, 0],
                                                  "07-12 months": [0, 0, 0],
                                                  "13-24 months": [0, 0, 0],
                                                  "25+ months": [0, 0, 0],
                                                  "Total": [0, 0, 0]
                                                  })
        return agyw_prevI_results_final

    def datim_agyw_prevII(self):
        __PERIOD_DATIM = sorted(list(self.__data.month_in_program_range.unique()))
        __PERIOD_DATIM.append("Total")
        __AGE_DATIM = sorted(list(self.__data.age_range.unique()))[0:3]
        try:
            pivotableII = self.__agyw_prevII.rename(
                columns={"age_range": "Age", "month_in_program_range": "Time"})
            agyw_prevII_pivot = pivotableII.pivot_table(
                index="Age", columns="Time", values="id_patient", aggfunc="count", fill_value=0, margins=True, margins_name="Total", dropna=False)[:-1]
            columns_pivotII = list(agyw_prevII_pivot.columns)
            indexes_pivotII = list(agyw_prevII_pivot.index)
            for period in __PERIOD_DATIM:
                if period not in columns_pivotII:
                    agyw_prevII_pivot[period] = 0
            for age in __AGE_DATIM:
                if age not in indexes_pivotII:
                    agyw_prevII_pivot.loc[age] = 0
            agyw_prevII_pivot = agyw_prevII_pivot.reindex(
                index=__AGE_DATIM, columns=__PERIOD_DATIM)
            agyw_prevII_pivot_final = agyw_prevII_pivot.reset_index().rename_axis(None, axis=1)
            agyw_prevII_results_final = DataFrame(
                agyw_prevII_pivot_final.to_records(index=False))
        except ValueError:
            agyw_prevII_results_final = DataFrame({"Age": ["10-14", "15-19",
                                                           "20-24"],
                                                   "0-6 months": [0, 0, 0],
                                                   "07-12 months": [0, 0, 0],
                                                   "13-24 months": [0, 0, 0],
                                                   "25+ months": [0, 0, 0],
                                                   "Total": [0, 0, 0]
                                                   })
        return agyw_prevII_results_final

    def datim_agyw_prevIII(self):
        __PERIOD_DATIM = sorted(list(self.__data.month_in_program_range.unique()))
        __PERIOD_DATIM.append("Total")
        __AGE_DATIM = sorted(list(self.__data.age_range.unique()))[0:3]
        try:
            pivotableIII = self.__agyw_prevIII.rename(
                columns={"age_range": "Age", "month_in_program_range": "Time"})
            agyw_prevIII_pivot = pivotableIII.pivot_table(
                index="Age", columns="Time", values="id_patient", aggfunc="count", fill_value=0, margins=True, margins_name="Total", dropna=False)[:-1]
            columns_pivotIII = list(agyw_prevIII_pivot.columns)
            indexes_pivotIII = list(agyw_prevIII_pivot.index)
            for period in __PERIOD_DATIM:
                if period not in columns_pivotIII:
                    agyw_prevIII_pivot[period] = 0
            for age in __AGE_DATIM:
                if age not in indexes_pivotIII:
                    agyw_prevIII_pivot.loc[age] = 0
            agyw_prevIII_pivot = agyw_prevIII_pivot.reindex(
                index=__AGE_DATIM, columns=__PERIOD_DATIM)
            agyw_prevIII_pivot_final = agyw_prevIII_pivot.reset_index().rename_axis(None, axis=1)
            agyw_prevIII_results_final = DataFrame(
                agyw_prevIII_pivot_final.to_records(index=False))
        except ValueError:
            agyw_prevIII_results_final = DataFrame({"Age": ["10-14",                     "15-19",
                                                            "20-24"],
                                                    "0-6 months": [0, 0, 0],
                                                    "07-12 months": [0, 0, 0],
                                                    "13-24 months": [0, 0, 0],
                                                    "25+ months": [0, 0, 0],
                                                    "Total": [0, 0, 0]
                                                    })
        return agyw_prevIII_results_final

    def datim_agyw_prevIV(self):
        __PERIOD_DATIM = sorted(list(self.__data.month_in_program_range.unique()))
        __PERIOD_DATIM.append("Total")
        __AGE_DATIM = sorted(list(self.__data.age_range.unique()))[0:3]
        try:
            pivotableIV = self.__agyw_prevIV.rename(
                columns={"age_range": "Age", "month_in_program_range": "Time"})
            agyw_prevIV_pivot = pivotableIV.pivot_table(
                index="Age", columns="Time", values="id_patient", aggfunc="count", fill_value=0, margins=True, margins_name="Total", dropna=False)[:-1]
            columns_pivotIII = list(agyw_prevIV_pivot.columns)
            indexes_pivotIII = list(agyw_prevIV_pivot.index)
            for period in __PERIOD_DATIM:
                if period not in columns_pivotIII:
                    agyw_prevIV_pivot[period] = 0
            for age in __AGE_DATIM:
                if age not in indexes_pivotIII:
                    agyw_prevIV_pivot.loc[age] = 0
            agyw_prevIV_pivot = agyw_prevIV_pivot.reindex(
                index=__AGE_DATIM, columns=__PERIOD_DATIM)
            agyw_prevIV_pivot_final = agyw_prevIV_pivot.reset_index().rename_axis(None, axis=1)
            agyw_prevIV_results_final = DataFrame(
                agyw_prevIV_pivot_final.to_records(index=False))
        except ValueError:
            agyw_prevIV_results_final = DataFrame({"Age": ["10-14",                     "15-19",
                                                           "20-24"],
                                                   "0-6 months": [0, 0, 0],
                                                   "07-12 months": [0, 0, 0],
                                                   "13-24 months": [0, 0, 0],
                                                   "25+ months": [0, 0, 0],
                                                   "Total": [0, 0, 0]
                                                   })
        return agyw_prevIV_results_final


datim = AgywPrev(data=AGYW_ACTIF)
datim


/var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/ipykernel_54471/1646952947.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.__dreams_valid["primary_only"] = self.__dreams_valid.apply(
/var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/ipykernel_54471/1646952947.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.__dreams_valid["primary_and_OneSecondary_services"] = self.__dreams_valid.apply(
/var/folders/7v/rs0k237d6nj3vdxfc1h7xtcw0000gn/T/ipykernel_54471/1646952947.py:49: SettingWithCopyWarni

<AgywPrev DATIM>

In [6]:
datim.data_agyw_prevI


,id_patient,id_parenting_group,departement,commune,nbre_pres_for_inter,nbre_parenting_coupe_present,has_comdom_topic,has_preventive_vbg,number_of_condoms_sensibilize,number_condoms_sensibilization_date_in_the_interval,...,prep,contraceptive_awareness,contraceptive,ps_1014,ps_1519,ps_2024,primary_only,primary_and_OneSecondary_services,completed_one_service,has_started_one_service
9399,121722,NaN,Ouest,Port-au-Prince,21,0,yes,yes,2,2,...,no,no,no,no,no,primary,full_primary_only,invalid,invalid,no
9400,121727,NaN,Ouest,Port-au-Prince,21,0,yes,yes,2,2,...,no,no,no,no,primary,no,full_primary_only,invalid,invalid,no
12090,170355,NaN,Artibonite,Saint-Marc,21,0,yes,yes,1,1,...,no,no,no,no,primary,no,full_primary_only,invalid,invalid,no
12091,170356,NaN,Artibonite,Saint-Marc,18,0,yes,yes,1,1,...,no,no,no,no,primary,no,full_primary_only,invalid,invalid,no
12092,170357,NaN,Artibonite,Saint-Marc,21,0,yes,yes,1,1,...,no,no,no,no,primary,no,full_primary_only,invalid,invalid,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49644,176240,NaN,Ouest,Tabarre,18,0,yes,yes,0,0,...,no,no,no,no,primary,no,full_primary_only,invalid,invalid,no
49645,176241,NaN,Ouest,Tabarre,18,0,yes,yes,0,0,...,no,no,no,no,primary,no,full_primary_only,invalid,invalid,no
49647,176243,NaN,Ouest,Tabarre,18,0,yes,yes,0,0,...,no,no,no,no,primary,no,full_primary_only,invalid,invalid,no
49651,176247,NaN,Ouest,Tabarre,18,0,yes,yes,0,0,...,no,no,no,no,no,primary,full_primary_only,invalid,invalid,no


In [7]:
datim.data_agyw_prevII


,id_patient,id_parenting_group,departement,commune,nbre_pres_for_inter,nbre_parenting_coupe_present,has_comdom_topic,has_preventive_vbg,number_of_condoms_sensibilize,number_condoms_sensibilization_date_in_the_interval,...,prep,contraceptive_awareness,contraceptive,ps_1014,ps_1519,ps_2024,primary_only,primary_and_OneSecondary_services,completed_one_service,has_started_one_service
226,75394,3363.0,Nord,Milot,21,10,yes,yes,0,0,...,no,no,no,no,primary,no,invalid,full_primary_leastOneSecondary,invalid,no
227,75395,NaN,Nord,Milot,21,0,yes,yes,0,0,...,no,no,no,no,primary,no,invalid,full_primary_leastOneSecondary,invalid,no
244,75528,NaN,Nord,Cap-Haïtien,17,0,yes,yes,1,0,...,no,no,no,no,no,primary,invalid,full_primary_leastOneSecondary,invalid,no
275,75987,NaN,Artibonite,Dessalines,17,0,yes,yes,1,0,...,no,no,no,no,no,primary,invalid,full_primary_leastOneSecondary,invalid,no
287,75964,NaN,Artibonite,Liancourt,17,0,yes,yes,0,0,...,no,no,no,no,no,primary,invalid,full_primary_leastOneSecondary,invalid,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49643,176239,NaN,Ouest,Port-au-Prince,18,0,yes,yes,0,0,...,no,no,no,primary,no,no,invalid,full_primary_leastOneSecondary,invalid,no
49646,176242,NaN,Ouest,Tabarre,18,0,yes,yes,0,0,...,no,no,no,primary,no,no,invalid,full_primary_leastOneSecondary,invalid,no
49648,176244,NaN,Ouest,Tabarre,18,0,yes,yes,0,0,...,no,no,no,primary,no,no,invalid,full_primary_leastOneSecondary,invalid,no
49649,176245,NaN,Ouest,Tabarre,18,0,yes,yes,0,0,...,no,no,no,primary,no,no,invalid,full_primary_leastOneSecondary,invalid,no


In [8]:
datim.data_agyw_prevIII


,id_patient,id_parenting_group,departement,commune,nbre_pres_for_inter,nbre_parenting_coupe_present,has_comdom_topic,has_preventive_vbg,number_of_condoms_sensibilize,number_condoms_sensibilization_date_in_the_interval,...,prep,contraceptive_awareness,contraceptive,ps_1014,ps_1519,ps_2024,primary_only,primary_and_OneSecondary_services,completed_one_service,has_started_one_service
1016,77682,NaN,Artibonite,Saint-Marc,16,0,yes,yes,0,0,...,no,no,no,no,no,no,invalid,invalid,primary_part_services,no
1098,79390,NaN,Ouest,Kenscoff,16,0,yes,yes,0,0,...,no,no,no,no,no,no,invalid,invalid,primary_part_services,no
1328,83020,NaN,Artibonite,Grande Saline,16,0,yes,yes,0,0,...,no,no,no,no,no,no,invalid,invalid,primary_part_services,no
1661,81162,NaN,Artibonite,Petite Rivière de l'Artibonite,12,0,yes,no,1,1,...,no,no,no,no,no,no,invalid,invalid,primary_part_services,no
1697,85286,NaN,Artibonite,Grande Saline,16,0,yes,yes,0,0,...,no,no,no,no,no,no,invalid,invalid,primary_part_services,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49664,101428,NaN,Nord,Milot,0,0,no,None,0,0,...,no,no,no,no,no,no,invalid,invalid,primary_part_services,no
49665,104847,NaN,Artibonite,Desdunes,0,0,no,None,0,0,...,no,no,no,no,no,no,invalid,invalid,primary_part_services,no
49666,105142,NaN,Nord,Cap-Haïtien,0,0,no,None,0,0,...,no,no,no,no,no,no,invalid,invalid,primary_part_services,no
49667,105158,NaN,Nord,Cap-Haïtien,0,0,no,None,0,0,...,no,no,no,no,no,no,invalid,invalid,primary_part_services,no


In [9]:
datim.data_agyw_prevIV

,id_patient,id_parenting_group,departement,commune,nbre_pres_for_inter,nbre_parenting_coupe_present,has_comdom_topic,has_preventive_vbg,number_of_condoms_sensibilize,number_condoms_sensibilization_date_in_the_interval,...,prep,contraceptive_awareness,contraceptive,ps_1014,ps_1519,ps_2024,primary_only,primary_and_OneSecondary_services,completed_one_service,has_started_one_service
46970,172134,NaN,Artibonite,Dessalines,11,0,no,yes,0,0,...,no,no,no,no,no,no,invalid,invalid,invalid,yes
47179,172626,NaN,Artibonite,Saint-Marc,6,0,no,no,0,0,...,no,no,no,no,no,no,invalid,invalid,invalid,yes
47180,172627,NaN,Artibonite,Saint-Marc,6,0,no,no,0,0,...,no,no,no,no,no,no,invalid,invalid,invalid,yes
47181,172628,NaN,Artibonite,Saint-Marc,6,0,no,no,0,0,...,no,no,no,no,no,no,invalid,invalid,invalid,yes
47182,172629,NaN,Artibonite,Saint-Marc,6,0,no,no,0,0,...,no,no,no,no,no,no,invalid,invalid,invalid,yes
47183,172630,NaN,Artibonite,Saint-Marc,6,0,no,no,0,0,...,no,no,no,no,no,no,invalid,invalid,invalid,yes
47184,172631,NaN,Artibonite,Saint-Marc,6,0,no,no,0,0,...,no,no,no,no,no,no,invalid,invalid,invalid,yes
47236,173126,NaN,Ouest,Tabarre,1,0,no,no,0,0,...,no,no,no,no,no,no,invalid,invalid,invalid,yes
47922,173917,NaN,Artibonite,Dessalines,7,0,no,yes,0,0,...,no,no,no,no,no,no,invalid,invalid,invalid,yes
48082,174201,NaN,Artibonite,Dessalines,10,0,no,yes,0,0,...,no,no,no,no,no,no,invalid,invalid,invalid,yes


In [10]:
datim.datim_agyw_prevI()

,Age,0-6 months,07-12 months,13-24 months,25+ months,Total
0,10-14,0,0,0,0,0
1,15-19,691,370,210,135,1406
2,20-24,188,145,82,110,525
